In [1]:
import csv
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from scipy import random
from numpy.linalg import pinv
from numpy.linalg import inv

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from timeit import default_timer as timer

random.seed(0)
from sklearn import preprocessing

path = "../datasets/"


numLags = 100
predictionStep = 1

nDimInput = numLags
nDimOutput = 1
lamb=0.0001
outputWeightFF = 0.92 

In [2]:
def getTimeEmbeddedMatrix(sequence, numLags=100, predictionStep=1):
  inDim = numLags
  X = np.zeros(shape=(len(sequence), inDim))
  T = np.zeros(shape=(len(sequence), 1))
  for i in range(numLags-1, len(sequence)-predictionStep):
    X[i, :] = np.array(sequence[columna][(i-numLags+1):(i+1)])
    T[i, :] = sequence[columna][i+predictionStep]
  return (X, T)

def linear(features,weights,bias):
   return np.dot(features, np.transpose(weights)) + bias

def sigmoidActFunc(features):
    return 1.0 / (1.0 + np.exp(-features))


def reluActFunc(features):
  return np.maximum(0,features)


def computeSquareDeviation(predictions, truth):
  squareDeviation = np.square(predictions-truth)
  return squareDeviation


class OSELM(object):
  def __init__(self, inputs, outputs, numHiddenNeurons, forgettingFactor=0.999,act = "sigmoid"):
    self.name = 'OSELM'
    self.inputs = inputs
    self.outputs = outputs
    self.numHiddenNeurons = numHiddenNeurons
    self.act = act
    # input to hidden weights
    self.inputWeights = None
    # bias of hidden units
    self.bias = None
    # hidden to output layer connection
    self.beta = None
    # auxiliary matrix used for sequential learning
    self.M = None

    self.forgettingFactor = forgettingFactor

  def calculateHiddenLayerActivation(self, features):
    """
    Calculate activation level of the hidden layer
    :param features feature matrix with dimension (numSamples, numInputs)
    :return: activation level (numSamples, numHiddenNeurons)
    """
    V = linear(features, self.inputWeights,self.bias)
    if(self.act == "sigmoid"):
        return sigmoidActFunc(V)
    if(self.act == "relu"):
        return reluActFunc(V)


  def initializePhase(self, lamb=0.0001):
    """
    Step 1: Initialization phase
    """
    # randomly initialize the input->hidden connections
    self.inputWeights = np.random.random((self.numHiddenNeurons, self.inputs))
    self.inputWeights = self.inputWeights * 2 - 1
    self.bias = np.random.random((1, self.numHiddenNeurons)) * 2 - 1
    # auxiliary matrix used for sequential learning
    self.M = inv(lamb*np.eye(self.numHiddenNeurons))
    # hidden to output layer connection
    self.beta = np.zeros([self.numHiddenNeurons,self.outputs])



  def train(self, features, targets):
    """
    Step 2: Sequential learning phase
    :param features feature matrix with dimension (numSamples, numInputs)
    :param targets target matrix with dimension (numSamples, numOutputs)
    """
    (numSamples, numOutputs) = targets.shape
    assert features.shape[0] == targets.shape[0]

    H = self.calculateHiddenLayerActivation(features)
    Ht = np.transpose(H)


    self.M = (1/self.forgettingFactor) * self.M - np.dot((1/self.forgettingFactor) * self.M,
                                     np.dot(Ht, np.dot(
                                       pinv(np.eye(numSamples) + np.dot(H, np.dot((1/self.forgettingFactor) * self.M, Ht))),
                                       np.dot(H, (1/self.forgettingFactor) * self.M))))
    self.beta = self.beta + np.dot(self.M, np.dot(Ht, targets - np.dot(H, self.beta)))

  def predict(self, features):
    """
    Make prediction with feature matrix
    :param features: feature matrix with dimension (numSamples, numInputs)
    :return: predictions with dimension (numSamples, numOutputs)
    """
    H = self.calculateHiddenLayerActivation(features)
    prediction = np.dot(H, self.beta)
    return prediction


def TrainModel(columna='none',net = None):
#     meanSeq = np.mean(df[columna])
#     stdSeq = np.std(df[columna])
#     df[columna] = (df[columna] - meanSeq)/stdSeq
    predictions= []
    target= []
    (X, T) = getTimeEmbeddedMatrix(df, numLags, predictionStep)
    for i in range(numLags, len(df)-predictionStep-1):
        net.train(X[[i], :], T[[i], :])
        Y = net.predict(X[[i+1], :])
        predictions.append(Y[0][0])
        target.append(T[i][0])
    predictions = np.array(predictions)
    target = np.array(target)
    
#     predictions = predictions * stdSeq + meanSeq
#     target = target * stdSeq + meanSeq
    #Calculate NRMSE from skip_eval to the end
    skip_eval=100
    squareDeviation = computeSquareDeviation(predictions, target)
    squareDeviation[:skip_eval] = None
    nrmse = np.sqrt(np.nanmean(squareDeviation)) / np.nanstd(predictions)
    #print("NRMSE {}".format(nrmse))
    predictions = np.array(predictions)
    target = np.array(target)
#     predictions = predictions * stdSeq + meanSeq
#     target = target * stdSeq + meanSeq
    
    return nrmse,predictions,target

In [3]:


dataset = "nyc_taxi.csv"
columna = 'passenger_count'

df = pd.read_csv(path + dataset, header=0, skiprows=[1])


for j in [25,50,100]:
    t_ = []
    errors = []
    for i in range(1):

        net = OSELM(inputs=nDimInput,outputs=nDimOutput, numHiddenNeurons=j,forgettingFactor=outputWeightFF,act = "sigmoid")
        net.initializePhase(lamb=lamb)

        start = timer()
        nrmse,predictions,target = TrainModel(columna=columna,net = net)
        end = timer()
        t_.append(end - start)
        errors.append(nrmse)
    print("\nNeurons =",j,"SIGMOID")
    print("Tiempo promedio",np.mean(t_))
    print("NRMSE promedio",np.mean(errors))
    

for j in [25,50,100]:
    t_ = []
    errors = []
    for i in range(1):

        net = OSELM(inputs=nDimInput,outputs=nDimOutput, numHiddenNeurons=j,forgettingFactor=outputWeightFF,act = "relu")
        net.initializePhase(lamb=lamb)

        start = timer()
        nrmse,predictions,target = TrainModel(columna=columna,net = net)
        end = timer()
        t_.append(end - start)
        errors.append(nrmse)
    print("\nNeurons =",j,"RELU")
    print("Tiempo promedio",np.mean(t_))
    print("NRMSE promedio",np.mean(errors))

C:\Users\Claudio\AppData\Local\Temp/ipykernel_32028/2555127852.py:14: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1.0 + np.exp(-features))
C:\Users\Claudio\AppData\Local\Temp/ipykernel_32028/2555127852.py:85: RuntimeWarning: overflow encountered in multiply
  self.M = (1/self.forgettingFactor) * self.M - np.dot((1/self.forgettingFactor) * self.M,
C:\Users\Claudio\AppData\Local\Temp/ipykernel_32028/2555127852.py:87: RuntimeWarning: overflow encountered in multiply
  pinv(np.eye(numSamples) + np.dot(H, np.dot((1/self.forgettingFactor) * self.M, Ht))),
C:\Users\Claudio\AppData\Local\Temp/ipykernel_32028/2555127852.py:88: RuntimeWarning: overflow encountered in multiply
  np.dot(H, (1/self.forgettingFactor) * self.M))))
C:\Users\Claudio\AppData\Local\Temp/ipykernel_32028/2555127852.py:22: RuntimeWarning: overflow encountered in square
  squareDeviation = np.square(predictions-truth)
c:\users\claudio\appdata\local\programs\python\python39\lib\site-packages\numpy\lib\nanfunct


Neurons = 25 SIGMOID
Tiempo promedio 2.223659099999992
NRMSE promedio nan

Neurons = 50 SIGMOID
Tiempo promedio 2.5328757999999993
NRMSE promedio nan

Neurons = 100 SIGMOID
Tiempo promedio 7.837563500000002
NRMSE promedio nan

Neurons = 25 RELU
Tiempo promedio 2.081320500000004
NRMSE promedio 1.0060649315777561

Neurons = 50 RELU
Tiempo promedio 2.3204404000000096
NRMSE promedio 1.0071782407569758

Neurons = 100 RELU
Tiempo promedio 7.377039999999994
NRMSE promedio nan


In [4]:
#ABALONE DATASET

dataset = "Abalone.csv"
columna = 'rings'

df = pd.read_csv(path + dataset, header=0, skiprows=[1])
map = {'M':0,'F':1,'I':2}
df = df.replace({'sex':map})

for j in [25,50,100]:
    t_ = []
    errors = []
    for i in range(1):

        net = OSELM(inputs=nDimInput,outputs=nDimOutput, numHiddenNeurons=j,forgettingFactor=outputWeightFF,act = "sigmoid")
        net.initializePhase(lamb=lamb)

        start = timer()
        nrmse,predictions,target = TrainModel(columna=columna,net = net)
        end = timer()
        t_.append(end - start)
        errors.append(nrmse)
    print("\nNeurons =",j,"SIGMOID")
    print("Tiempo promedio",np.mean(t_))
    print("NRMSE promedio",np.mean(errors))
    

for j in [25,50,100]:
    t_ = []
    errors = []
    for i in range(1):

        net = OSELM(inputs=nDimInput,outputs=nDimOutput, numHiddenNeurons=j,forgettingFactor=outputWeightFF,act = "relu")
        net.initializePhase(lamb=lamb)

        start = timer()
        nrmse,predictions,target = TrainModel(columna=columna,net = net)
        end = timer()
        t_.append(end - start)
        errors.append(nrmse)
    print("\nNeurons =",j,"RELU")
    print("Tiempo promedio",np.mean(t_))
    print("NRMSE promedio",np.mean(errors))

C:\Users\Claudio\AppData\Local\Temp/ipykernel_11436/2555127852.py:22: RuntimeWarning: overflow encountered in square
  squareDeviation = np.square(predictions-truth)
C:\Users\Claudio\AppData\Local\Temp/ipykernel_11436/2555127852.py:123: RuntimeWarning: invalid value encountered in double_scalars
  nrmse = np.sqrt(np.nanmean(squareDeviation)) / np.nanstd(predictions)



Neurons = 25 SIGMOID
Tiempo promedio 0.49280070000000364
NRMSE promedio nan

Neurons = 50 SIGMOID
Tiempo promedio 0.5621091000000007
NRMSE promedio nan

Neurons = 100 SIGMOID
Tiempo promedio 2.107155700000007
NRMSE promedio nan

Neurons = 25 RELU
Tiempo promedio 0.48705640000000017
NRMSE promedio 1.0125813877336751

Neurons = 50 RELU
Tiempo promedio 0.5529568999999981
NRMSE promedio 1.012609583807863

Neurons = 100 RELU
Tiempo promedio 2.0956309999999974
NRMSE promedio 1.0127850775369605


In [5]:
#AUTO MPG DATASET

dataset = "auto-mpg.csv"
columna = 'mpg'

df = pd.read_csv(path + dataset, header=0, skiprows=[1])
df.drop(['car name'],inplace=True,axis=1)


for j in [25,50,100]:
    t_ = []
    errors = []
    for i in range(1):

        net = OSELM(inputs=nDimInput,outputs=nDimOutput, numHiddenNeurons=j,forgettingFactor=outputWeightFF,act = "sigmoid")
        net.initializePhase(lamb=lamb)

        start = timer()
        nrmse,predictions,target = TrainModel(columna=columna,net = net)
        end = timer()
        t_.append(end - start)
        errors.append(nrmse)
    print("\nNeurons =",j,"SIGMOID")
    print("Tiempo promedio",np.mean(t_))
    print("NRMSE promedio",np.mean(errors))
    

for j in [25,50,100]:
    t_ = []
    errors = []
    for i in range(50):

        net = OSELM(inputs=nDimInput,outputs=nDimOutput, numHiddenNeurons=j,forgettingFactor=outputWeightFF,act = "relu")
        net.initializePhase(lamb=lamb)

        start = timer()
        nrmse,predictions,target = TrainModel(columna=columna,net = net)
        end = timer()
        t_.append(end - start)
        errors.append(nrmse)
    print("\nNeurons =",j,"RELU")
    print("Tiempo promedio",np.mean(t_))
    print("NRMSE promedio",np.mean(errors))


Neurons = 25 SIGMOID
Tiempo promedio 0.039117199999992636
NRMSE promedio 1.1748785704667803

Neurons = 50 SIGMOID
Tiempo promedio 0.04184120000000746
NRMSE promedio 1.232071417550871

Neurons = 100 SIGMOID
Tiempo promedio 0.1379612999999864
NRMSE promedio 1.2321958515006823

Neurons = 25 RELU
Tiempo promedio 0.03540395599999897
NRMSE promedio 0.7002001251063998

Neurons = 50 RELU
Tiempo promedio 0.039751057999999374
NRMSE promedio 0.7206753007956977

Neurons = 100 RELU
Tiempo promedio 0.13854475999999977
NRMSE promedio 0.8440854613060216


In [6]:
#COVID CHILE DATASET
dataset = "covid_chile.csv"
columna = 'confirmed'

df = pd.read_csv(path + dataset, header=0, skiprows=[1])
df["day"] = [t.day for t in pd.DatetimeIndex(df.date)]
df["month"] = [t.month for t in pd.DatetimeIndex(df.date)]
df.drop(["date"],inplace=True,axis=1)

for j in [25,50,100]:
    t_ = []
    errors = []
    for i in range(1):

        net = OSELM(inputs=nDimInput,outputs=nDimOutput, numHiddenNeurons=j,forgettingFactor=outputWeightFF,act = "sigmoid")
        net.initializePhase(lamb=lamb)

        start = timer()
        nrmse,predictions,target = TrainModel(columna=columna,net = net)
        end = timer()
        t_.append(end - start)
        errors.append(nrmse)
    print("\nNeurons =",j,"SIGMOID")
    print("Tiempo promedio",np.mean(t_))
    print("NRMSE promedio",np.mean(errors))
    

for j in [25,50,100]:
    t_ = []
    errors = []
    for i in range(50):

        net = OSELM(inputs=nDimInput,outputs=nDimOutput, numHiddenNeurons=j,forgettingFactor=outputWeightFF,act = "relu")
        net.initializePhase(lamb=lamb)

        start = timer()
        nrmse,predictions,target = TrainModel(columna=columna,net = net)
        end = timer()
        t_.append(end - start)
        errors.append(nrmse)
    print("\nNeurons =",j,"RELU")
    print("Tiempo promedio",np.mean(t_))
    print("NRMSE promedio",np.mean(errors))

C:\Users\Claudio\AppData\Local\Temp/ipykernel_11436/2555127852.py:14: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1.0 + np.exp(-features))



Neurons = 25 SIGMOID
Tiempo promedio 0.048896899999988364
NRMSE promedio 1.1876787815749261

Neurons = 50 SIGMOID
Tiempo promedio 0.05151619999998047
NRMSE promedio 1.1877192017074565

Neurons = 100 SIGMOID
Tiempo promedio 0.17528909999998632
NRMSE promedio 1.1876708402636074

Neurons = 25 RELU
Tiempo promedio 0.04081189800000061
NRMSE promedio 0.02396440418830622

Neurons = 50 RELU
Tiempo promedio 0.04724798200000123
NRMSE promedio 0.0423716862250668


C:\Users\Claudio\AppData\Local\Temp/ipykernel_11436/2555127852.py:22: RuntimeWarning: overflow encountered in square
  squareDeviation = np.square(predictions-truth)
c:\users\claudio\appdata\local\programs\python\python39\lib\site-packages\numpy\lib\nanfunctions.py:1544: RuntimeWarning: overflow encountered in multiply
  sqr = np.multiply(arr, arr, out=arr)
c:\users\claudio\appdata\local\programs\python\python39\lib\site-packages\numpy\core\fromnumeric.py:87: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
C:\Users\Claudio\AppData\Local\Temp/ipykernel_11436/2555127852.py:123: RuntimeWarning: invalid value encountered in double_scalars
  nrmse = np.sqrt(np.nanmean(squareDeviation)) / np.nanstd(predictions)



Neurons = 100 RELU
Tiempo promedio 0.16705700999999692
NRMSE promedio nan


In [7]:
#Google Stock Price DATASET
dataset = "Google_Stock_Price_Train.csv"
columna = 'Open'

df = pd.read_csv(path + dataset, header=0, skiprows=[1])
df["day"] = [t.day for t in pd.DatetimeIndex(df.Date)]
df["month"] = [t.month for t in pd.DatetimeIndex(df.Date)]
df['year'] = [t.year for t in pd.DatetimeIndex(df.Date)]
df.drop(['Date',"day","month"],inplace=True,axis=1)

df['Volume'] = df['Volume'].str.replace(',', '')
df['Close'] = df['Close'].str.replace(',', '')


for j in [25,50,100]:
    t_ = []
    errors = []
    for i in range(50):

        net = OSELM(inputs=nDimInput,outputs=nDimOutput, numHiddenNeurons=j,forgettingFactor=outputWeightFF,act = "sigmoid")
        net.initializePhase(lamb=lamb)

        start = timer()
        nrmse,predictions,target = TrainModel(columna=columna,net = net)
        end = timer()
        t_.append(end - start)
        errors.append(nrmse)
    print("\nNeurons =",j,"SIGMOID")
    print("Tiempo promedio",np.mean(t_))
    print("NRMSE promedio",np.mean(errors))
    

for j in [25,50,100]:
    t_ = []
    errors = []
    for i in range(50):

        net = OSELM(inputs=nDimInput,outputs=nDimOutput, numHiddenNeurons=j,forgettingFactor=outputWeightFF,act = "relu")
        net.initializePhase(lamb=lamb)

        start = timer()
        nrmse,predictions,target = TrainModel(columna=columna,net = net)
        end = timer()
        t_.append(end - start)
        errors.append(nrmse)
    print("\nNeurons =",j,"RELU")
    print("Tiempo promedio",np.mean(t_))
    print("NRMSE promedio",np.mean(errors))

C:\Users\Claudio\AppData\Local\Temp/ipykernel_11436/2555127852.py:14: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1.0 + np.exp(-features))
C:\Users\Claudio\AppData\Local\Temp/ipykernel_11436/2555127852.py:22: RuntimeWarning: overflow encountered in square
  squareDeviation = np.square(predictions-truth)
c:\users\claudio\appdata\local\programs\python\python39\lib\site-packages\numpy\lib\nanfunctions.py:1544: RuntimeWarning: overflow encountered in multiply
  sqr = np.multiply(arr, arr, out=arr)
C:\Users\Claudio\AppData\Local\Temp/ipykernel_11436/2555127852.py:123: RuntimeWarning: invalid value encountered in double_scalars
  nrmse = np.sqrt(np.nanmean(squareDeviation)) / np.nanstd(predictions)
c:\users\claudio\appdata\local\programs\python\python39\lib\site-packages\numpy\lib\nanfunctions.py:1539: RuntimeWarning: invalid value encountered in subtract
  np.subtract(arr, avg, out=arr, casting='unsafe')



Neurons = 25 SIGMOID
Tiempo promedio 0.1531960979999974
NRMSE promedio nan

Neurons = 50 SIGMOID
Tiempo promedio 0.1721858759999992
NRMSE promedio nan

Neurons = 100 SIGMOID
Tiempo promedio 0.5943146399999978
NRMSE promedio nan

Neurons = 25 RELU
Tiempo promedio 0.1439560199999994
NRMSE promedio 0.22475655807608863

Neurons = 50 RELU
Tiempo promedio 0.17293387800000004
NRMSE promedio 0.4264760572424708

Neurons = 100 RELU
Tiempo promedio 0.5952463639999996
NRMSE promedio 0.6594382454092169


In [8]:
#REC CENTER DATASET
dataset = "rec-center-hourly.csv"
columna = 'kw_energy_consumption'

df = pd.read_csv(path + dataset, header=0, skiprows=[1,2])

df["day"] = [t.day for t in pd.DatetimeIndex(df.timestamp)]
df["month"] = [t.month for t in pd.DatetimeIndex(df.timestamp)]
df['year'] = [t.year for t in pd.DatetimeIndex(df.timestamp)]
df.drop(['timestamp'],inplace=True,axis=1)


for j in [25,50,100]:
    t_ = []
    errors = []
    for i in range(20):

        net = OSELM(inputs=nDimInput,outputs=nDimOutput, numHiddenNeurons=j,forgettingFactor=outputWeightFF,act = "sigmoid")
        net.initializePhase(lamb=lamb)

        start = timer()
        nrmse,predictions,target = TrainModel(columna=columna,net = net)
        end = timer()
        t_.append(end - start)
        errors.append(nrmse)
    print("\nNeurons =",j,"SIGMOID")
    print("Tiempo promedio",np.mean(t_))
    print("NRMSE promedio",np.mean(errors))
    

for j in [25,50,100]:
    t_ = []
    errors = []
    for i in range(20):

        net = OSELM(inputs=nDimInput,outputs=nDimOutput, numHiddenNeurons=j,forgettingFactor=outputWeightFF,act = "relu")
        net.initializePhase(lamb=lamb)

        start = timer()
        nrmse,predictions,target = TrainModel(columna=columna,net = net)
        end = timer()
        t_.append(end - start)
        errors.append(nrmse)
    print("\nNeurons =",j,"RELU")
    print("Tiempo promedio",np.mean(t_))
    print("NRMSE promedio",np.mean(errors))

C:\Users\Claudio\AppData\Local\Temp/ipykernel_11436/2555127852.py:14: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1.0 + np.exp(-features))
C:\Users\Claudio\AppData\Local\Temp/ipykernel_11436/2555127852.py:22: RuntimeWarning: overflow encountered in square
  squareDeviation = np.square(predictions-truth)
c:\users\claudio\appdata\local\programs\python\python39\lib\site-packages\numpy\lib\nanfunctions.py:1544: RuntimeWarning: overflow encountered in multiply
  sqr = np.multiply(arr, arr, out=arr)
C:\Users\Claudio\AppData\Local\Temp/ipykernel_11436/2555127852.py:123: RuntimeWarning: invalid value encountered in double_scalars
  nrmse = np.sqrt(np.nanmean(squareDeviation)) / np.nanstd(predictions)



Neurons = 25 SIGMOID
Tiempo promedio 0.5512935300000066
NRMSE promedio nan

Neurons = 50 SIGMOID
Tiempo promedio 0.6317611399999976
NRMSE promedio nan


c:\users\claudio\appdata\local\programs\python\python39\lib\site-packages\numpy\lib\nanfunctions.py:1539: RuntimeWarning: invalid value encountered in subtract
  np.subtract(arr, avg, out=arr, casting='unsafe')



Neurons = 100 SIGMOID
Tiempo promedio 2.1767473849999988
NRMSE promedio nan

Neurons = 25 RELU
Tiempo promedio 0.5338925100000097
NRMSE promedio nan

Neurons = 50 RELU
Tiempo promedio 0.605705279999998
NRMSE promedio nan


c:\users\claudio\appdata\local\programs\python\python39\lib\site-packages\numpy\core\fromnumeric.py:87: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)



Neurons = 100 RELU
Tiempo promedio 2.2002208449999956
NRMSE promedio nan


In [9]:
dataset = "train_bikeDemand.csv"
columna = 'count'

df = pd.read_csv(path + dataset, header=0, skiprows=[1,2])
df = df.drop(df.columns[0], axis=1)

for j in [25,50,100]:
    t_ = []
    errors = []
    for i in range(20):

        net = OSELM(inputs=nDimInput,outputs=nDimOutput, numHiddenNeurons=j,forgettingFactor=outputWeightFF,act = "sigmoid")
        net.initializePhase(lamb=lamb)

        start = timer()
        nrmse,predictions,target = TrainModel(columna=columna,net = net)
        end = timer()
        t_.append(end - start)
        errors.append(nrmse)
    print("\nNeurons =",j,"SIGMOID")
    print("Tiempo promedio",np.mean(t_))
    print("NRMSE promedio",np.mean(errors))
    

for j in [25,50,100]:
    t_ = []
    errors = []
    for i in range(20):

        net = OSELM(inputs=nDimInput,outputs=nDimOutput, numHiddenNeurons=j,forgettingFactor=outputWeightFF,act = "relu")
        net.initializePhase(lamb=lamb)

        start = timer()
        nrmse,predictions,target = TrainModel(columna=columna,net = net)
        end = timer()
        t_.append(end - start)
        errors.append(nrmse)
    print("\nNeurons =",j,"RELU")
    print("Tiempo promedio",np.mean(t_))
    print("NRMSE promedio",np.mean(errors))

C:\Users\Claudio\AppData\Local\Temp/ipykernel_11436/2555127852.py:14: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1.0 + np.exp(-features))
C:\Users\Claudio\AppData\Local\Temp/ipykernel_11436/2555127852.py:22: RuntimeWarning: overflow encountered in square
  squareDeviation = np.square(predictions-truth)
C:\Users\Claudio\AppData\Local\Temp/ipykernel_11436/2555127852.py:123: RuntimeWarning: invalid value encountered in double_scalars
  nrmse = np.sqrt(np.nanmean(squareDeviation)) / np.nanstd(predictions)
C:\Users\Claudio\AppData\Local\Temp/ipykernel_11436/2555127852.py:85: RuntimeWarning: overflow encountered in multiply
  self.M = (1/self.forgettingFactor) * self.M - np.dot((1/self.forgettingFactor) * self.M,
C:\Users\Claudio\AppData\Local\Temp/ipykernel_11436/2555127852.py:87: RuntimeWarning: overflow encountered in multiply
  pinv(np.eye(numSamples) + np.dot(H, np.dot((1/self.forgettingFactor) * self.M, Ht))),
C:\Users\Claudio\AppData\Local\Temp/ipykernel_11436/2555127


Neurons = 25 SIGMOID
Tiempo promedio 1.552766299999999
NRMSE promedio nan

Neurons = 50 SIGMOID
Tiempo promedio 1.6892369200000048
NRMSE promedio nan

Neurons = 100 SIGMOID
Tiempo promedio 5.839234034999981
NRMSE promedio nan

Neurons = 25 RELU
Tiempo promedio 1.3820061100000032
NRMSE promedio nan

Neurons = 50 RELU
Tiempo promedio 1.557581230000011
NRMSE promedio nan

Neurons = 100 RELU
Tiempo promedio 5.6506778149999946
NRMSE promedio nan


In [10]:
dataset = 'filtered_avocados.csv'
avocados = pd.read_csv(path + dataset)
new_columns = pd.DataFrame(avocados["Date"].str.split('-').tolist(), columns = ['yr', 'month', 'day'])


new_columns["day"] = new_columns["day"].astype(np.int32)
new_columns["month"] = new_columns["month"].astype(np.int32)

avocados["day"] = new_columns["day"]
avocados["month"] = new_columns["month"]
avocados["type"]=pd.Categorical(avocados["type"])
avocados["region"]=pd.Categorical(avocados["region"])
df_type = pd.get_dummies(avocados['type'], prefix = 'category')
df_region = pd.get_dummies(avocados['region'], prefix = 'category')


del avocados["Date"]
del avocados["type"]
del avocados["region"]
avocados_cleaned=pd.concat([avocados, df_type, df_region], axis=1, sort=False)

df = avocados_cleaned
columna = 'AveragePrice'


for j in [25,50,100]:
    t_ = []
    errors = []
    for i in range(20):

        net = OSELM(inputs=nDimInput,outputs=nDimOutput, numHiddenNeurons=j,forgettingFactor=outputWeightFF,act = "sigmoid")
        net.initializePhase(lamb=lamb)

        start = timer()
        nrmse,predictions,target = TrainModel(columna=columna,net = net)
        end = timer()
        t_.append(end - start)
        errors.append(nrmse)
    print("\nNeurons =",j,"SIGMOID")
    print("Tiempo promedio",np.mean(t_))
    print("NRMSE promedio",np.mean(errors))
    

for j in [25,50,100]:
    t_ = []
    errors = []
    for i in range(20):

        net = OSELM(inputs=nDimInput,outputs=nDimOutput, numHiddenNeurons=j,forgettingFactor=outputWeightFF,act = "relu")
        net.initializePhase(lamb=lamb)

        start = timer()
        nrmse,predictions,target = TrainModel(columna=columna,net = net)
        end = timer()
        t_.append(end - start)
        errors.append(nrmse)
    print("\nNeurons =",j,"RELU")
    print("Tiempo promedio",np.mean(t_))
    print("NRMSE promedio",np.mean(errors))

C:\Users\Claudio\AppData\Local\Temp/ipykernel_11436/2555127852.py:22: RuntimeWarning: overflow encountered in square
  squareDeviation = np.square(predictions-truth)
C:\Users\Claudio\AppData\Local\Temp/ipykernel_11436/2555127852.py:123: RuntimeWarning: invalid value encountered in double_scalars
  nrmse = np.sqrt(np.nanmean(squareDeviation)) / np.nanstd(predictions)



Neurons = 25 SIGMOID
Tiempo promedio 0.9356724650000501
NRMSE promedio nan

Neurons = 50 SIGMOID
Tiempo promedio 1.07424089000001
NRMSE promedio nan

Neurons = 100 SIGMOID
Tiempo promedio 3.78871886999998
NRMSE promedio nan

Neurons = 25 RELU
Tiempo promedio 0.9216924999999719
NRMSE promedio 1.0262284826928967

Neurons = 50 RELU
Tiempo promedio 1.0334734900000284
NRMSE promedio nan

Neurons = 100 RELU
Tiempo promedio 3.730274650000024
NRMSE promedio nan
